# Проект: мэтчинг товаров

**Описание проекта:** в данном проекте рассматривается решение задачи matching - сопоставления. С помощью машинного обучения необходимо определить похожие товары, размещенные на маркетплейсе.

**Цель:** разработать модель, решающую задачу мэтчинга товаров.

**Исходные данные:** обучающий датасет, валидационный датасет и base (датасет со всеми товарами) содержат индексы товаров из base, обезличенные признаки товаров и “правильные ответы” - индексы наиболее подходящих товаров-аналогов (кроме валидационного датасета - для него “правильные ответы” содержатся в отдельном файле).

## Загрузка данных

In [1]:
!apt -q install libomp-dev
!pip -q install faiss-gpu
!pip install annoy
!pip install phik

Reading package lists...
Building dependency tree...
Reading state information...
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik
import statsmodels.api as sm
import math
import statistics as stat
import faiss
from tqdm.notebook import tqdm
import requests
import os
from urllib.parse import urlencode
from annoy import AnnoyIndex
import zipfile
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    OrdinalEncoder,
    MinMaxScaler,
    RobustScaler
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [3]:
# Функция для загрузки данных
def downloader(size: str='small'):
    if size not in ['small', 'large']:
        raise Exception('Unknown Argument')
    elif size == 'small':
        public_key = 'https://disk.yandex.ru/d/YQElc_cNQQLSOw'
    else:
        public_key = 'https://disk.yandex.ru/d/BBEphK0EHSJ5Jw'

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open('/content/data.zip', 'wb') as f:
        f.write(download_response.content)

    zip_path = ('/content/data.zip')

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()

In [4]:
def plot_hist_box(df, y):
    """
    Функция для построения гистограмм и бокс-плотов для числовых колонок в датафрейме pandas.

    Параметры:
    df (DataFrame) - датафрейм pandas, содержащий числовые колонки для анализа.
    y (str) - имя колонки, по которой будет выполнена группировка данных.

    Вывод:
    - Выводит гистограмму и бокс-плот для каждой числовой колонки в датафрейме, сгруппированные по значению колонки y.
    - Гистограммы показывают распределение значений числовых колонок, а бокс-плоты - их распределение и наличие выбросов.

    В случае, если в датафрейме нет числовых колонок, функция выводит сообщение и завершает работу.
    """
    # определение числовых колонок
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

    # исключение колонки 'id', если она есть в числовых колонках
    if 'id' in numeric_columns:
        numeric_columns = numeric_columns.drop('id')

    # если нет числовых колонок, выход из функции
    if len(numeric_columns) == 0:
        print("Нет числовых колонок для отображения.")
        return

    # формирование полотна для графиков
    m = len(numeric_columns)
    n = 2  # два графика для каждой числовой переменной
    fig, axes = plt.subplots(m, n, figsize=(15, 5*m))

    # если есть всего одна числовая колонка, решаем проблему одномерного массива
    if len(numeric_columns) == 1:
        for i, column in enumerate(numeric_columns):
            # гистограмма
            sns.histplot(data=df, x=column, kde=True, hue=y, ax=axes[i])
            axes[i].set_title(f'Гистограмма {column}')

            # boxplot
            sns.boxplot(data=df, x=df[column], y=y, ax=axes[i + len(numeric_columns)])
            axes[i + len(numeric_columns)].set_title(f'Боксплот {column}')
    else:

        # построение графиков
        for i, column in enumerate(numeric_columns):
            # гистограмма
            sns.histplot(data=df, x=column, hue=y, kde=True, ax=axes[i, 0])
            axes[i, 0].set_title(f'Гистограмма {column}')

            # boxplot
            sns.boxplot(data=df, x=df[column], y=y, ax=axes[i, 1])
            axes[i, 1].set_title(f'Боксплот {column}')

    # регулировка расположения графиков
    plt.tight_layout()
    plt.show()


In [5]:
%%time
downloader('small') # загружаем уменьшенную версию данных

CPU times: user 3.37 s, sys: 1.08 s, total: 4.45 s
Wall time: 14.4 s


In [6]:
%%time
base = pd.read_csv("base.csv", index_col=0) # описание всех товаров
train = pd.read_csv("train.csv", index_col=0) # тренировочная выборка
validation = pd.read_csv("validation.csv", index_col=0) # тестовая выборка
validation_answer = pd.read_csv("validation_answer.csv", index_col=0) # верные ответы к тестовой выборке

CPU times: user 7.3 s, sys: 460 ms, total: 7.76 s
Wall time: 11.5 s


### Просмотр данных

In [7]:
base.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4207931-base,-43.946243,15.364378,17.515854,-132.31146,157.064420,-4.069252,-340.630860,-57.550140,128.398220,45.090958,-126.84374,4.494522,-99.842310,44.926903,177.52173,-12.291790,38.470360,105.357650,-142.46024,-80.163260,-110.368935,1047.517357,-69.594620,66.313540,84.873870,813.770071,-81.038780,16.162964,-98.244880,159.53406,27.554913,-209.184280,62.05977,-529.295053,114.598330,90.469894,-20.256914,-164.76800,-133.31387,-41.252960,-10.251193,8.289038,-131.312710,75.704500,-16.483078,40.771038,-146.096740,-143.407680,49.807987,63.434480,-30.250080,20.470263,78.079910,-128.915310,92.327680,63.885570,-141.17464,142.902590,-93.068596,-568.421584,-90.018690,-129.015670,-71.927170,30.711966,-90.190475,-24.931271,66.972534,106.346634,-44.270622,155.98834,-1074.464888,-25.066608
2710972-base,-73.004890,4.923342,-19.750746,-136.52908,99.907170,-70.709110,-567.401996,-128.890150,109.914986,201.472200,-186.22650,29.896042,-99.770996,0.126302,136.19049,-35.224740,-30.321323,-43.148834,-162.85175,-79.714510,-75.784870,1507.231274,-69.654564,43.640663,-4.779669,813.770071,43.976913,11.924875,-50.228523,166.00820,-59.505333,-115.332520,72.18324,-735.671365,96.322300,85.796360,-22.030330,-147.54501,-108.38295,-45.084892,-15.004004,-1.532826,-46.456585,197.578950,-56.199876,60.298710,-102.653340,-108.967964,58.512012,-9.678028,-85.448300,-68.686080,71.590200,-232.425690,91.706856,63.290657,-137.33595,-47.124687,-148.057400,-543.787056,-160.651600,-133.462220,-109.044660,20.916021,-171.201390,-110.596844,67.730100,8.909615,-9.470253,133.29536,-545.897014,-72.913230
1371460-base,-85.565570,-0.493598,-48.374817,-157.98502,96.809510,-81.710210,-22.297688,79.768670,124.357086,105.715180,-149.80756,-54.501680,-21.037973,-24.887660,128.38864,-58.558483,34.862656,19.784412,-130.91820,-79.032230,-166.635250,1507.231274,-8.495993,61.205086,25.895348,813.770071,-140.768860,20.872790,-123.957570,126.34781,11.713674,-125.025154,152.68590,-1018.469545,-22.444600,73.897640,9.190645,-156.51881,-92.18573,-34.926760,-13.277475,16.026424,-33.853546,119.604520,-52.525341,71.204750,-178.702940,-88.278500,30.501453,16.651737,-88.377014,-55.883583,70.182980,-89.233925,92.005780,76.458725,-131.14087,40.914352,-157.900540,-394.319235,-87.107025,-120.772545,-58.821650,41.369606,-132.934500,-43.016839,67.871925,141.778240,69.048520,111.72038,-1111.038833,-23.087206
3438601-base,-105.564090,15.393871,-46.223934,-158.11488,79.514114,-48.944480,-93.713010,38.581398,123.397960,110.324326,-161.18800,-68.519790,-0.607330,38.733696,120.74344,-14.109269,28.868027,-29.858810,-94.30395,-79.339810,-138.984270,1507.231274,-131.885380,70.031360,32.736595,813.770071,-62.370860,13.763219,-31.872276,139.55270,9.836465,-150.221130,80.14020,-537.183707,3.091667,129.699330,-63.429424,-169.02724,-119.77007,-28.637785,-8.315162,2.752385,-160.293820,85.086890,-18.251750,90.374054,1.479935,-121.983050,65.852660,8.355225,34.118896,-57.069756,70.461800,-127.905410,94.314280,71.259940,-135.57787,-39.982346,-159.751560,-230.147648,-95.221160,-148.814090,-87.907290,-58.806870,-147.794800,-155.830237,68.974754,21.397510,126.098785,139.73320,-1282.707248,-74.527940
422798-base,-74.638880,11.315012,-40.204174,-161.76430,50.507114,-80.775560,-640.923467,65.225000,122.344940,191.465850,-156.98384,-76.650210,-75.674970,12.624029,145.33752,-35.774258,11.598761,-11.460761,-201.35443,-77.779366,-120.968400,548.736883,19.851685,17.943344,27.063320,813.770071,-85.483780,21.236433,-95.071020,132.61092,13.526038,-160.476840,104.71937,-304.174382,-15.385452,91.418655,-36.474556,-157.43959,-102.83162,-56.782710,-19.969252,-0.598189,-222.228790,33.441666,-56.092110,71.276030,-8.713509,-86.099380,8.488903,-14.959278,86.812996,-29.6

In [8]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291813 entries, 4207931-base to 274130-base
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       291813 non-null  float64
 1   1       291813 non-null  float64
 2   2       291813 non-null  float64
 3   3       291813 non-null  float64
 4   4       291813 non-null  float64
 5   5       291813 non-null  float64
 6   6       291813 non-null  float64
 7   7       291813 non-null  float64
 8   8       291813 non-null  float64
 9   9       291813 non-null  float64
 10  10      291813 non-null  float64
 11  11      291813 non-null  float64
 12  12      291813 non-null  float64
 13  13      291813 non-null  float64
 14  14      291813 non-null  float64
 15  15      291813 non-null  float64
 16  16      291813 non-null  float64
 17  17      291813 non-null  float64
 18  18      291813 non-null  float64
 19  19      291813 non-null  float64
 20  20      291813 non-null  float64
 21 

In [9]:
base.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
count,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000
mean,-86.274741,8.078087,-44.613480,-146.605552,111.261183,-71.875015,-393.430460,20.391975,123.676692,124.427158,-143.241671,-69.943874,-41.180997,7.608319,148.940275,-23.195314,-0.939536,-19.230876,-137.380758,-79.189608,-98.276602,1257.119646,-6.817302,56.737918,26.645141,726.539553,-65.999767,16.297703,-86.279764,151.737398,-11.569343,-144.233906,87.271323,-555.030333,36.354556,75.165481,-16.450743,-146.667125,-118.525117,-41.902869,-9.505951,4.069182,-112.666843,120.820683,-39.788570,58.962295,-85.102851,-138.542656,47.962621,-32.597303,26.591666,-21.808728,67.033070,-117.973309,88.033431,69.206822,-132.891610,13.104575,-119.628076,-315.824414,-64.894401,-130.688468,-79.075207,33.244836,-154.720293,13.699277,67.792659,23.543147,74.890028,115.611366,-798.355219,-47.701336
std,24.918947,4.949495,38.545928,19.842726,46.350083,28.189743,272.084921,64.297194,6.352832,64.366631,41.828261,51.194297,38.942859,36.576025,33.623142,13.078519,20.826134,52.952607,46.676018,0.757619,28.155715,400.140140,51.984937,34.382587,38.963153,206.009752,61.551614,4.657027,48.329281,10.093783,28.934056,34.905297,27.781245,260.728908,39.894835,21.457381,25.832185,9.952702,13.125440,19.208804,8.188480,5.829764,56.950392,46.780878,14.735848,32.027228,63.265200,57.718288,45.199092,58.254867,59.066918,51.778506,13.063570,54.774973,4.773332,11.955396,7.820257,51.070116,55.163448,210.631518,62.525113,9.573147,30.485074,28.896144,41.217568,98.996416,1.825081,55.353032,61.320347,21.219222,385.414338,41.731025
min,-189.356020,-12.594500,-231.785920,-224.880500,-95.240830,-188.473330,-791.468770,-296.171050,93.684616,-143.499600,-321.522580,-367.130700,-215.223530,-175.623890,-9.157745,-84.724250,-96.368160,-237.018780,-348.336240,-82.659546,-230.840270,136.855507,-219.015870,-83.704820,-143.183290,-61.967366,-327.226200,-7.296732,-302.772400,107.650450,-146.495420,-288.613980,-25.102974,-1045.311529,-126.765594,-8.800842,-125.226540,-190.385990,-168.637420,-130.044040,-44.640232,-19.545673,-335.376220,-78.077320,-63.102401,-71.691635,-334.213750,-389.026640,-133.092030,-322.812070,-245.635030,-264.633450,16.126598,-361.489350,66.851030,18.996754,-165.421420,-196.183610,-328.829280,-681.042024,-400.791100,-169.028370,-214.821140,-85.825500,-346.239320,-157.593866,59.835790,-213.492420,-190.483150,18.601448,-1297.924962,-209.935760
25%,-103.154300,4.709408,-69.535900,-159.882740,80.367640,-91.227936,-631.937855,-22.085905,119.473625,81.685610,-170.579130,-103.997734,-66.559790,-16.890121,126.042564,-31.799664,-15.135077,-55.462906,-167.724230,-79.680220,-116.912186,1086.240670,-42.276604,33.351940,0.418633,813.770071,-107.548615,13.250520,-118.420650,144.899320,-30.690517,-167.123320,68.438130,-771.296708,9.454500,60.934887,-34.117690,-153.286560,-127.443540,-5

In [10]:
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
109249-query,-24.021454,3.122524,-80.947525,-112.329994,191.090180,-66.903130,-759.626065,-75.284454,120.55149,131.131700,-149.211060,-102.31221,21.387623,11.277594,143.22140,-22.011570,-3.618249,-16.005480,-133.38228,-78.89356,-65.690530,407.773575,-11.660624,67.008150,24.975033,813.770071,40.051064,17.933155,-75.435745,149.81720,-23.413877,-178.095570,133.786470,-906.571061,113.355560,83.942260,-16.592659,-146.52074,-120.237860,-27.341612,-8.845615,1.027612,-175.647720,167.735820,-32.931559,47.860960,-196.247500,-118.81005,-4.762772,-114.877680,37.397278,-55.616966,56.627056,-108.43317,87.372560,76.513430,-136.27057,3.652915,-164.574510,-635.284275,-75.647255,-116.679340,-41.234684,-24.601670,-167.760770,133.678516,68.184600,26.317545,11.938202,148.549320,-778.563381,-46.877750,66971-base
34137-query,-82.033580,8.115866,-8.793022,-182.972100,56.645336,-52.597610,-55.720337,130.059250,129.38335,76.202880,-137.799420,33.30165,-2.868191,-34.318770,189.06479,-19.337550,-14.208210,-71.110245,-157.74814,-78.70069,-91.741875,1054.205600,-41.845630,102.128620,72.559050,813.770071,-37.957787,17.598982,-159.975400,140.02528,-8.819328,-147.055180,113.819870,-529.295053,70.674940,55.976795,8.817799,-134.14812,-73.679794,-57.566544,-4.338496,-3.270682,-144.499200,144.650200,-37.903276,58.913525,-105.362840,-125.66783,19.367283,-29.087658,-35.021350,26.627962,55.718437,-110.52611,83.513374,75.926130,-135.68242,-7.429803,-180.645020,11.470171,16.464691,-121.807236,-90.814450,54.448433,-120.894806,-12.292085,66.608116,-27.997612,10.091335,95.809265,-1022.691531,-88.564705,1433819-base
136121-query,-75.719640,-0.223386,-86.186130,-162.064060,114.320114,-53.394600,-117.261013,-24.857851,124.80780,112.190155,-200.925960,-38.86518,-80.611270,14.343805,156.62129,-22.498169,-26.359468,-109.034870,-106.92659,-79.74731,-69.876830,1507.231274,-20.058287,34.334927,23.592144,813.770071,-49.503860,22.166200,-85.740160,134.83647,-69.569850,-139.887240,67.377045,-341.781842,54.161224,81.891660,36.421352,-159.99583,-131.916080,-20.495195,-13.976569,-2.355247,-216.228650,238.836490,-56.611536,43.366640,7.191841,-159.48369,-19.338009,-51.409897,36.819540,32.536880,80.681020,-232.40741,84.053690,59.086180,-139.85950,78.409440,-115.940575,2.426572,7.594826,-126.520134,-73.148960,-5.609123,-93.029880,-80.997871,63.733383,11.378683,62.932007,130.975390,-1074.464888,-74.861176,290133-base
105191-query,-56.580620,5.093593,-46.943110,-149.039120,112.436430,-76.820510,-324.995645,-32.833107,119.47865,120.074790,-61.347084,-28.67060,-102.790180,-36.194320,157.18976,-33.318240,7.448413,-47.230713,-178.04608,-78.78652,-106.235440,1507.231274,-63.414307,38.099255,-89.795350,813.770071,-107.432390,10.052701,-71.917380,147.74005,-18.750763,-143.795620,67.207310,-366.139446,112.187700,78.144810,-41.085410,-132.75719,-89.445030,-19.267069,-14.866466,7.775788,-104.302110,74.622894,-59.875136,76.406470,-77.797020,-92.01658,19.337300,-37.922787,37.271270,111.639570,94.912950,-179.72540,86.601480,62.698364,-122.16293,29.873940,-53.508120,-0.938894,-36.919907,-144.555000,-96.798590,21.624313,-158.880370,179.597294,69.891360,-33.804955,233.914610,122.868546,-1074.464888,-93.775375,1270048-base
63983-query,-52.725650,9.027046,-92.829650,-113.111010,134.124970,-42.423073,-759.626065,8.261169,119.49023,172.365360,-186.641390,-84.94380,-92.339966,-30.229528,167.86163,-22.635653,0.014536,-9.796367,-213.10180,-78.59006,-98.728300,1250.423749,-43.892487,86.288450,-1.549826,813.770071,-110.356980,24.055641,-96.578270,156.58230,45.124240,-123.888504,118.035110,-607.946912,52.311410,76.747800,-14.161914,-143.53851,-124.886215,-64.783330,-17.706848,15.446568,-53.554455,174.381620,-23.140892,76.419330,-73.

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9999 entries, 109249-query to 13504-query
Data columns (total 73 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       9999 non-null   float64
 1   1       9999 non-null   float64
 2   2       9999 non-null   float64
 3   3       9999 non-null   float64
 4   4       9999 non-null   float64
 5   5       9999 non-null   float64
 6   6       9999 non-null   float64
 7   7       9999 non-null   float64
 8   8       9999 non-null   float64
 9   9       9999 non-null   float64
 10  10      9999 non-null   float64
 11  11      9999 non-null   float64
 12  12      9999 non-null   float64
 13  13      9999 non-null   float64
 14  14      9999 non-null   float64
 15  15      9999 non-null   float64
 16  16      9999 non-null   float64
 17  17      9999 non-null   float64
 18  18      9999 non-null   float64
 19  19      9999 non-null   float64
 20  20      9999 non-null   float64
 21  21      9999 non-null   

In [12]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000
mean,-85.469569,7.608015,-43.886495,-146.204146,111.979577,-73.679417,-443.890756,21.820551,123.107400,126.418041,-140.639126,-69.334526,-42.286946,5.943081,149.113455,-23.937223,-1.952591,-13.265594,-138.188847,-79.180062,-97.464981,1299.001685,-3.538495,56.512052,23.047551,741.740801,-64.888281,16.298482,-86.455233,152.169915,-13.677299,-144.461507,89.452082,-585.522842,36.489419,73.109286,-13.966612,-146.521017,-118.313406,-41.205789,-9.363432,4.183629,-104.602746,122.199398,-40.440898,59.493747,-83.718482,-141.372371,45.601665,-34.662104,30.469524,-23.010338,67.467159,-120.629620,88.182224,69.498609,-132.557160,14.617704,-119.977907,-314.115197,-66.974786,-130.515897,-81.373935,37.137381,-152.473803,16.533799,67.803505,23.750006,73.254974,115.410518,-708.824875,-48.325315
std,25.986217,4.985728,39.318615,20.309633,47.599872,28.743810,281.494323,66.450440,6.418815,64.858194,41.384532,51.226487,40.946886,37.525080,34.054664,13.015724,20.950774,54.944036,48.968389,0.777864,28.254012,371.742755,50.212678,35.451912,41.189893,193.164266,63.593672,4.813034,48.752131,10.352239,29.424935,36.344429,28.544929,273.876582,39.165112,19.798149,25.525332,10.249859,13.653104,19.884643,8.329687,5.937301,55.886895,48.254306,14.845848,30.871626,66.194787,59.715662,45.974134,59.539827,60.657572,55.721563,13.076620,56.003612,4.768590,12.430994,7.979449,49.141112,55.661756,209.602864,64.483125,9.370190,30.628554,25.391546,41.109706,98.956198,1.875665,54.838217,61.459193,21.641422,405.686503,40.899658
min,-173.032560,-11.560507,-187.292630,-219.724800,-71.243350,-184.962450,-791.443909,-250.681340,99.001770,-113.104600,-287.202640,-344.520570,-201.855180,-128.040480,21.577286,-77.179040,-70.050766,-222.327650,-329.597720,-82.252945,-228.985530,136.900447,-174.593440,-75.980484,-120.500656,-61.640878,-318.192700,-4.305775,-269.041630,114.099030,-125.440050,-261.522860,-12.177368,-1043.159721,-112.779755,-1.458771,-108.400100,-188.973650,-163.109950,-123.062420,-43.014280,-16.524162,-307.125950,-73.530136,-63.098819,-40.429813,-297.294860,-364.509220,-108.377380,-251.944990,-217.258000,-216.835040,21.089088,-321.383970,70.048590,22.668636,-161.124900,-134.082200,-318.879640,-681.033307,-301.453370,-167.331150,-191.308230,-60.515068,-306.837650,-157.579209,61.339855,-165.361250,-127.017555,34.696030,-1297.871984,-208.276810
25%,-103.445763,4.178606,-69.990780,-160.032880,79.712390,-93.620305,-757.628755,-21.344349,118.787718,81.959692,-168.315630,-103.138320,-70.147210,-19.432253,126.442184,-32.289482,-16.294177,-49.630930,-171.156920,-79.681540,-115.316515,1248.126198,-37.116690,32.634457,-5.998924,813.770071,-106.627325,13.131956,-119.623572,145.391380,-32.950590,-169.025685,69.572715,-803.390708,9.817245,60.199024,-31.081269,-153.293210,-127.522585,-54.564564,-14.941572,0.084014,-141.943035,88.563728,-54.117874,38.387535,-129.638000,-181.844230,14.723253,-73.511487,-9.889670,-60.958610,58.2

**Всего данные содержат 72 признака, все из которых являются обезличенными и приведены к типу float, кроме того распределения признаков различны. В тренировочной выборке 10 000 наблюдений, а всего товаров в датасете base 291 813**

### Подготовка данных

In [13]:
# Проверка на дубли
base.duplicated().sum()

11278

In [14]:
# Удаление дублей
base.drop_duplicates(inplace=True)

In [15]:
# Определяем столбцы с пропущенными значениями
columns_with_missing = base.columns[base.isnull().any()].tolist()

# Выводим названия столбцов и количество пропусков
if len(columns_with_missing) == 0:
    print('Пропущенных значений нет')
else:
    for column in columns_with_missing:
        missing_count = base[column].isnull().sum()
        print(f"Столбец '{column}' имеет {missing_count} пропущенных значений.")

Пропущенных значений нет


In [16]:
# Проверка на дубли
train.duplicated().sum()

243

In [17]:
# Удаление дублей
train.drop_duplicates(inplace=True)

In [18]:
# Определяем столбцы с пропущенными значениями
columns_with_missing = train.columns[train.isnull().any()].tolist()

# Выводим названия столбцов и количество пропусков
if len(columns_with_missing) == 0:
    print('Пропущенных значений нет')
else:
    for column in columns_with_missing:
        missing_count = train[column].isnull().sum()
        print(f"Столбец '{column}' имеет {missing_count} пропущенных значений.")

Пропущенных значений нет


In [19]:
plot_hist_box(train, None)

**В данных нет пропусков, однако обнаружены дубликаты, которые были удалены.** \
**Признаки распределены различным образом, чаще всего встречается распределение похожее на нормальное, признаки 21 и 25 имеют перекос в сторону одного бина, т.е. около 80% наблюдений имеют одно и тоже значение: это может быть как и "заглушка" на пропусках, так и реальное распределение данных. Однако в силу обезличенности данных интерпретировать это не представляется возможным. (Если бы мы знали что это такое, мы не знаем что это такое)**

### Поиск наилучшей модели

In [20]:
targets = train["Target"] # целевая переменная
train.drop("Target", axis=1, inplace=True) # признаки

In [21]:
def create_faiss_index(data, n_cells=20, nprobe=1): #n_trees=10,
    dimension = data.shape[1]
    quantizer = faiss.IndexFlatL2(dimension)
    index = faiss.IndexIVFFlat(quantizer, dimension, n_cells)
    index.nprobe = nprobe
    index.train(np.ascontiguousarray(data).astype('float32'))
    index.add(np.ascontiguousarray(data).astype('float32'))
    #index.add(data)
    return index

def search_faiss(index, train, candidate_number=5):
    r, idx = index.search(np.ascontiguousarray(train).astype('float32'), candidate_number)
    return r, idx

def evaluate_accuracy(idx, r, true_answer, base_index):
    acc = 0
    for target, el in zip(true_answer.values.tolist(), idx.tolist()):
      acc += int(target in [base_index[r] for r in el])
      accuracy = acc / len(idx)
    return accuracy

if __name__ == "__main__":

    # Перебор скалеров
    var_scalers = [StandardScaler(), MinMaxScaler()]
    # Перебор параметров индекса
    var_n_cells = [20, 30, 50]
    var_nprobe = [15, 20, 25]
    base_index = {k: v for k, v in enumerate(base.index.to_list())}

    best_accuracy = 0
    best_params = None


    for scaler in var_scalers:
        scaler = scaler
        scaled_base = scaler.fit_transform(base)
        scaled_train = scaler.transform(train)

        for n_cells in var_n_cells:
            for nprobe in var_nprobe:
                # Создание индекса FAISS
                faiss_index = create_faiss_index(scaled_base, n_cells=n_cells, nprobe=nprobe)

                # Поиск ближайших соседей
                r, idx = search_faiss(faiss_index, scaled_train, candidate_number=5)

                # Оценка точности
                faiss_accuracy = evaluate_accuracy(idx, r, targets, base_index)


                # Обновление лучшей точности и параметров
                if faiss_accuracy > best_accuracy:
                    best_accuracy = faiss_accuracy
                    best_params = {'scaler': scaler.__class__.__name__, 'n_cells': n_cells, 'nprobe': nprobe}

    print(f'Best accuracy: {best_accuracy:.1%} with parameters: {best_params}')


Best accuracy: 76.6% with parameters: {'scaler': 'StandardScaler', 'n_cells': 30, 'nprobe': 25}


#### Инициализация индекса с наилучшими параметрами

In [22]:
# Определяем размерность векторов. Это число указывает, сколько измерений имеет каждый вектор в наборе данных.
dims = base.shape[1]
# Количество ячеек, на которые будет разделено пространство векторов. Это число влияет на баланс между скоростью поиска и точностью результатов.
n_cells = best_params['n_cells']
# Создаем "пустой" индекс, который будет использоваться как квантёр (quantizer).
# Квантёр отвечает за определение, в какую ячейку должен попадать каждый вектор. В этом случае, поскольку индекс "пустой", он не содержит никаких векторов, но готов к добавлению векторов и поиску.
quantizer = faiss.IndexFlatL2(dims)
# Создаем IVF индекс. IVF-индекс организован таким образом, что пространство векторов разделяется на n_cells ячеек,
# и каждый вектор индексируется в соответствии с его положением в одном из этих пространств.
# Это позволяет значительно уменьшить количество необходимой памяти для хранения индексов, особенно при работе с большими наборами данных.
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [23]:
idx_l2.nprobe = best_params['nprobe']  # Проходим по топ-25 центроид для поиска top-5 ближайших соседей

#### Обучение IVF-индекса

In [24]:
# Масштабируем данные
scaler = StandardScaler()
scaled_base = scaler.fit_transform(base)
scaled_validation = scaler.transform(validation)

In [25]:
candidate_number = 5 # Сколько ближайщих соседей нужно найти

In [26]:
%%time
# Этот вызов обучает IVF-индекс. В процессе обучения индекс вычисляет центроиды для каждой из n_cells ячеек, на которые было разделено пространство векторов.
# Центроиды — это средние точки всех векторов, которые были добавлены в каждую ячейку.
# Обучение позволяет индексу лучше понять распределение данных и оптимизировать поиск ближайших соседей.
idx_l2.train(np.ascontiguousarray(scaled_base).astype('float32')) # np.ascontiguousarray(...) гарантирует, что массив передается в память в непрерывном порядке

# Этот вызов добавляет векторы в IVF-индекс. После обучения индекса, векторы могут быть добавлены в индекс, и индекс будет знать, в какую ячейку поместить каждый вектор.
# Это позволяет индексу эффективно хранить информацию о расположении векторов и быстро находить ближайших соседей.
idx_l2.add(np.ascontiguousarray(scaled_base).astype('float32'))

CPU times: user 583 ms, sys: 107 ms, total: 690 ms
Wall time: 512 ms


In [27]:
%%time
r, indx = idx_l2.search(np.ascontiguousarray(scaled_validation).astype('float32'), candidate_number)

CPU times: user 5min 28s, sys: 498 ms, total: 5min 28s
Wall time: 3min 22s


In [28]:
# Этот фрагмент кода создаёт словарь base_index, который ассоциирует индексы элементов из списка base.index.to_list() с их позиционными номерами в этом списке.
base_index = {k: v for k, v in enumerate(base.index.tolist())}

'''
Ускорение доступа к данным: Использование словаря вместо прямого обращения к элементам списка по индексу может ускорить процесс поиска,
поскольку доступ к элементам словаря осуществляется за константное время O(1), в отличие от списка, где доступ к элементу происходит за линейное время O(n).

'''

' \nУскорение доступа к данным: Использование словаря вместо прямого обращения к элементам списка по индексу может ускорить процесс поиска, \nпоскольку доступ к элементам словаря осуществляется за константное время O(1), в отличие от списка, где доступ к элементу происходит за линейное время O(n).\n\n'

### Расчет точности

In [29]:
# Инициализирует переменную acc значением 0. Эта переменная будет использоваться для подсчёта количества успешных поисков.
acc = 0
# Функция zip объединяет два списка: targets.values.tolist() содержит целевые значения, а idx.tolist() содержит индексы найденных ближайших соседей.
# Для каждого элемента в этих списках создаётся кортеж (target, el), где target — это целевое значение, а el — список индексов найденных соседей.
for target, el in zip(validation_answer.squeeze().tolist(), indx.tolist()):
    # [base_index[r] for r in el]: Создаёт список, содержащий значения из словаря base_index для каждого индекса r в списке el. Это позволяет получить целевые значения, соответствующие найденным индексам.
    # int(target in...): Проверяет, содержится ли целевое значение target в списке найденных целевых значений. Если да, то возвращает 1, если нет — 0. Это значение добавляется к общему счётчику acc.
    acc += int(target in [base_index[r] for r in el])

print(f'Accuracy @ {candidate_number} = {acc / len(indx):.1%}')

Accuracy @ 5 = 75.8%


**Модель FAISS с наилучшими параметрами (n_cells=30, nprobe=25) на тесте имеет результат точности 75.8%, то есть почти в 76 из 100 случаев, среди 5 подобранных моделью продуктов, имеется похожий продукт.**